# Componentes para Grafos No Dirigidos

## Descubriendo Componentes Conectados

In [ ]:
import networkx as nx
import numpy as np
import matplotlib.pyplot as plt
from networkx.algorithms import community

Utilizaremos el Grafo del Club de Karate de Zachary (1970)

In [ ]:
G = nx.karate_club_graph()
plt.figure(figsize=(16,9))
pos = nx.spring_layout(G, k=0.3)
nx.draw_networkx(G, pos)

Revisamos la información del Grafo

In [ ]:
print(nx.info(G))

¿Es una red conectada?

In [ ]:
nx.is_connected(G)

Cantidad de componentes conectados

In [ ]:
nx.number_connected_components(G)

In [ ]:
nx.has_bridges(G)

In [ ]:
list(nx.bridges(G))

In [ ]:
list(nx.local_bridges(G))

### Algoritmo de Clustering de Girvan Newman

In [ ]:
communities = community.centrality.girvan_newman(G)

In [ ]:
#tuple(sorted(c) for c in next(communities))

In [ ]:
node_groups = []
for com in next(communities):
    node_groups.append(list(com))
print(node_groups)

In [ ]:
plt.figure(figsize=(16,9))
color_map = []
for node in G:
    if node in node_groups[0]:
        color_map.append('b')
    else: 
        color_map.append('g')  
nx.draw_networkx(G, pos=pos, node_color=color_map, with_labels=True)
plt.show()

## Algoritmo Label Propagation

In [ ]:
compr=community.label_propagation_communities(G)

In [ ]:
colors = ["#00C98D", "#5030C0", "#50F0F0"]
color_map_b = ["black"] * nx.number_of_nodes(G)
counter = 0
for c in compr:
    for n in c:
        color_map_b[n] = colors[counter]
    counter = counter + 1
plt.figure(figsize=(16,9))
nx.draw_networkx_edges(G, pos)
nx.draw_networkx_nodes(G, pos, node_color=color_map_b)
nx.draw_networkx_labels(G, pos)
plt.show()

## Algoritmo Greedy Modularity

In [ ]:
comgm = community.greedy_modularity_communities(G)

In [ ]:
color_map_b = ["black"] * nx.number_of_nodes(G)
counter = 0
for c in comgm:
    for n in c:
        color_map_b[n] = colors[counter]
    counter = counter + 1
plt.figure(figsize=(16,9))
nx.draw_networkx_edges(G, pos)
nx.draw_networkx_nodes(G, pos, node_color=color_map_b)
nx.draw_networkx_labels(G, pos)
plt.show()

## Algoritmo de Kernighan-Lin
Es necesario fijar un enlace de inicio para empezar a recorrer el grafo

In [ ]:
init_nodes = np.array_split(G.nodes(), 2)
init_partition = [set(init_nodes[0]), set(init_nodes[1])]
print(init_partition)

In [ ]:
color_map_i = ["black"] * nx.number_of_nodes(G)
counter = 0
for c in init_partition:
    for n in c:
        color_map_i[n] = colors[counter]
    counter = counter +1
plt.figure(figsize=(16,9))
nx.draw_networkx_edges(G, pos)
nx.draw_networkx_nodes(G, pos, node_color=color_map_i)
nx.draw_networkx_labels(G, pos)
plt.show()

In [ ]:
comkl = community.kernighan_lin_bisection(G, partition=init_partition)

In [ ]:
color_map_b = ["black"] * nx.number_of_nodes(G)
counter = 0
for c in comkl:
    for n in c:
        color_map_b[n] = colors[counter]
    counter = counter + 1
plt.figure(figsize=(16,9))
nx.draw_networkx_edges(G, pos)
nx.draw_networkx_nodes(G, pos, node_color=color_map_b)
nx.draw_networkx_labels(G, pos)
plt.show()

# Red Desconectada con Pesos

Creamos una Red de tres componentes conectados

In [ ]:
G = nx.Graph()
G.add_weighted_edges_from([('A', 'B',5),('B', 'C',7),('C', 'A',2),('E', 'F',5),('D', 'E',4),('C', 'E',3),('D', 'C',1),('C', 'F',6)])
G.add_weighted_edges_from([('H', 'I',2),('I', 'J',1),('J', 'K',2),('H', 'J',3),('I', 'K',4),('H', 'K',2)])
G.add_weighted_edges_from([('M', 'N',3),('N', 'O',7),('N', 'P',4),('P', 'R',1),('P', 'Q',2),('Q', 'R',1),('O', 'P',5),('M', 'O',2),('M', 'R',1),('O', 'S',6)])
print(G.nodes(), G.edges())

In [ ]:
plt.figure(figsize=(16,9))
pos=nx.spring_layout(G)
nx.draw_networkx(G,pos)

In [ ]:
nx.is_connected(G)

In [ ]:
nx.number_connected_components(G)

Listar nodos que forman parte del componente conectado de un nodo específico

In [ ]:
nx.node_connected_component(G, 'C')

In [ ]:
nx.node_connected_component(G, 'P')

Lista de todos los componentes

In [ ]:
list(nx.connected_components(G))

Tamaño de cada componente conectado

In [ ]:
[len(c) for c in nx.connected_components(G) if len(c) > 2]

Extraer el grafo para cada componente conectado

In [ ]:
S = [G.subgraph(c).copy() for c in nx.connected_components(G)]
S

Identificar el componente conectado más grande

In [ ]:
largest_cc = max(nx.connected_components(G), key=len)
largest_cc

Explorar cada Subgrafo

In [ ]:
print(nx.info(S[2]))

In [ ]:
print(S[2].nodes(), S[2].edges())

Función para filtrar los enlaces que tienen un peso menor a un límite fijado por el argumento "weight"

In [ ]:
def trim_edges(g, weight=1):
        g2=nx.Graph()
        for f, to, edata in g.edges(data=True):
                if edata['weight'] > weight:
                        g2.add_edge(f,to,weight=edata)
        return g2

In [ ]:
G1=trim_edges(G,3)

In [ ]:
print(G1.nodes(), G1.edges())

Función del "Método de la Isla" para ir extrayendo componentes con distintos pesos mínimos

In [ ]:
def island_method(g, iterations=5):
    weights= [edata['weight'] for f,to,edata in g.edges(data=True)]
    mn=int(min(weights))
    mx=int(max(weights))
    #compute the size of the step, so we get a reasonable step in iterations
    step=int((mx-mn)/iterations)
    return [[threshold, trim_edges(g, threshold)] for threshold in range(mn,mx,step)]

Cada iteración representa un cambio en el valor mínimo de los enlaces 

In [ ]:
G2=island_method(G)
G2

Explorar los resultados considerando un peso mayor que 2

In [ ]:
print(G2[1][1].nodes(), G2[1][1].edges())

Explorar los resultados considerando un peso mayor que 5

In [ ]:
print(G2[4][1].nodes(), G2[4][1].edges())

También se puede aplicar el método de la isla al componente más grande, para reducir su tamaño y complejidad

In [ ]:
islands=island_method(S[2])

Print the threshold level, size of the graph, and number of connected components

In [ ]:
for i in islands:
    print(i[0], len(i[1]), nx.number_connected_components(i[1]))

### Algoritmo de Clustering de Girvan Newman

In [ ]:
communities = community.centrality.girvan_newman(G)

In [ ]:
#tuple(sorted(c) for c in next(communities))

In [ ]:
node_groups = []
for com in next(communities):
    node_groups.append(list(com))
print(node_groups)

In [ ]:
plt.figure(figsize=(16,9))
color_map = []
for node in G:
    if node in node_groups[0]:
        color_map.append('b')
    else:
        if node in node_groups[1]:
            color_map.append('g')
        else:
            if node in node_groups[2]:
                color_map.append('r')
            else:
                color_map.append('y')
nx.draw_networkx(G, pos=pos, node_color=color_map, with_labels=True)
plt.show()

Elaborado por Luis Cajachahua bajo licencia MIT (2021)